# Chapter 24: Introspection and Code Generation

Python's rich introspection capabilities let you examine objects, functions, and classes
at runtime. Combined with dynamic code generation tools like `type()`, `exec()`, and
the `ast` module, you can build powerful abstractions such as ORMs, validators, and
code analysis tools.

## Topics Covered
- **`inspect` module**: `signature()`, `getsource()`, `getmembers()`
- **`inspect` predicates**: `isclass()`, `isfunction()`, `ismethod()`
- **`inspect.Parameter`** and **`Signature`** objects
- **`type()`** as a class factory (3-argument form)
- **`exec()` and `eval()`**: Dynamic code execution and dangers
- **`ast` module**: Parsing and analyzing Python source
- **Code generation patterns**: Creating classes and functions at runtime
- **Practical**: Building a simple ORM-style field validator with introspection

## The `inspect` Module: Examining Objects at Runtime

The `inspect` module provides tools to examine live objects: retrieve source code,
inspect function signatures, list class members, and determine what kind of object
you are looking at.

In [ ]:
import inspect
from typing import Any


class SampleClass:
    """A sample class for introspection demos."""
    class_var: str = "shared"

    def __init__(self, x: int, y: int = 10) -> None:
        self.x = x
        self.y = y

    def add(self) -> int:
        """Return the sum of x and y."""
        return self.x + self.y

    @classmethod
    def from_tuple(cls, pair: tuple[int, int]) -> "SampleClass":
        return cls(pair[0], pair[1])

    @staticmethod
    def description() -> str:
        return "A sample class"

    @property
    def magnitude(self) -> float:
        return (self.x**2 + self.y**2) ** 0.5


# getsource: retrieve the actual source code
print("Source of SampleClass.add:")
print(inspect.getsource(SampleClass.add))

# getmembers: list all members with optional predicate filtering
print("Methods of SampleClass:")
methods = inspect.getmembers(SampleClass, predicate=inspect.isfunction)
for name, obj in methods:
    print(f"  {name}: {obj}")

## Inspect Predicates: Classifying Objects

The `inspect` module provides predicates to determine what kind of object you have.
These are essential when writing generic tools that need to handle different object
types differently.

In [ ]:
import inspect
import os


def sample_function(x: int) -> int:
    return x * 2


sample_lambda = lambda x: x * 2


class Example:
    def method(self) -> None:
        pass

    @classmethod
    def cls_method(cls) -> None:
        pass

    @staticmethod
    def static_method() -> None:
        pass


instance = Example()

# Test various predicates
objects_to_test: dict[str, object] = {
    "sample_function": sample_function,
    "sample_lambda": sample_lambda,
    "Example (class)": Example,
    "instance.method": instance.method,
    "os (module)": os,
    "42 (int)": 42,
}

predicates = [
    ("isfunction", inspect.isfunction),
    ("ismethod", inspect.ismethod),
    ("isclass", inspect.isclass),
    ("ismodule", inspect.ismodule),
    ("isbuiltin", inspect.isbuiltin),
]

# Print a truth table
header = f"{'Object':<22s}" + "".join(f"{name:<14s}" for name, _ in predicates)
print(header)
print("-" * len(header))

for label, obj in objects_to_test.items():
    row = f"{label:<22s}"
    for _, pred in predicates:
        row += f"{str(pred(obj)):<14s}"
    print(row)

## `inspect.Parameter` and `Signature` Objects

`inspect.signature()` returns a `Signature` object that provides structured access to
a callable's parameters: their names, kinds (positional, keyword, etc.), defaults,
and annotations. This is the foundation for building tools that work with arbitrary
function signatures.

In [ ]:
import inspect


def complex_function(
    pos_only: int,
    /,
    normal: str,
    default_val: float = 3.14,
    *args: int,
    keyword_only: bool = False,
    **kwargs: str,
) -> dict:
    """A function with every kind of parameter."""
    return {}


sig = inspect.signature(complex_function)
print(f"Signature: {sig}")
print(f"Return annotation: {sig.return_annotation}")

# Map parameter kind enum values to readable names
kind_names: dict[int, str] = {
    inspect.Parameter.POSITIONAL_ONLY: "POSITIONAL_ONLY",
    inspect.Parameter.POSITIONAL_OR_KEYWORD: "POSITIONAL_OR_KEYWORD",
    inspect.Parameter.VAR_POSITIONAL: "VAR_POSITIONAL (*args)",
    inspect.Parameter.KEYWORD_ONLY: "KEYWORD_ONLY",
    inspect.Parameter.VAR_KEYWORD: "VAR_KEYWORD (**kwargs)",
}

print(f"\n{'Name':<16s} {'Kind':<28s} {'Default':<16s} {'Annotation'}")
print("-" * 76)
for name, param in sig.parameters.items():
    default = (
        repr(param.default)
        if param.default is not inspect.Parameter.empty
        else "(none)"
    )
    annotation = (
        param.annotation.__name__
        if isinstance(param.annotation, type)
        else str(param.annotation)
        if param.annotation is not inspect.Parameter.empty
        else "(none)"
    )
    kind = kind_names.get(param.kind, str(param.kind))
    print(f"{name:<16s} {kind:<28s} {default:<16s} {annotation}")

In [ ]:
import inspect


# Practical use: build a signature-aware wrapper
def describe_callable(func: object) -> str:
    """Generate a human-readable description of a callable's signature."""
    sig = inspect.signature(func)  # type: ignore[arg-type]
    parts: list[str] = []

    required = []
    optional = []

    for name, param in sig.parameters.items():
        if param.kind in (param.VAR_POSITIONAL, param.VAR_KEYWORD):
            continue
        if param.default is inspect.Parameter.empty:
            required.append(name)
        else:
            optional.append(f"{name}={param.default!r}")

    parts.append(f"Required: {', '.join(required) or '(none)'}")
    parts.append(f"Optional: {', '.join(optional) or '(none)'}")
    parts.append(f"Accepts *args: {any(p.kind == p.VAR_POSITIONAL for p in sig.parameters.values())}")
    parts.append(f"Accepts **kwargs: {any(p.kind == p.VAR_KEYWORD for p in sig.parameters.values())}")

    return "\n".join(parts)


def example_func(host: str, port: int, *, ssl: bool = True, **options: str) -> None:
    pass


print(f"Describing: {inspect.signature(example_func)}")
print(describe_callable(example_func))

## `type()` as a Class Factory (3-Argument Form)

Beyond checking an object's type, `type()` has a 3-argument form that **creates new
classes** at runtime: `type(name, bases, namespace)`. This is exactly what Python does
behind the scenes when it encounters a `class` statement.

In [ ]:
from typing import Any


# Creating a class with the class statement
class DogStatic:
    species: str = "Canis familiaris"

    def __init__(self, name: str, age: int) -> None:
        self.name = name
        self.age = age

    def speak(self) -> str:
        return f"{self.name} says Woof!"

    def __repr__(self) -> str:
        return f"Dog({self.name!r}, age={self.age})"


# Creating the EXACT same class dynamically with type()
def dog_init(self: Any, name: str, age: int) -> None:
    self.name = name
    self.age = age


def dog_speak(self: Any) -> str:
    return f"{self.name} says Woof!"


def dog_repr(self: Any) -> str:
    return f"Dog({self.name!r}, age={self.age})"


DogDynamic = type(
    "DogDynamic",              # class name
    (),                         # base classes (tuple)
    {                           # namespace (class body as a dict)
        "species": "Canis familiaris",
        "__init__": dog_init,
        "speak": dog_speak,
        "__repr__": dog_repr,
    },
)

# Both classes work identically
for cls in [DogStatic, DogDynamic]:
    d = cls("Rex", 5)
    print(f"{cls.__name__}: {d!r}, {d.speak()}, species={d.species!r}")

print(f"\ntype(DogDynamic) = {type(DogDynamic)}")
print(f"isinstance check: {isinstance(DogDynamic('Spot', 3), DogDynamic)}")

In [ ]:
from typing import Any


# Practical use: a class factory function
def make_struct(name: str, fields: list[str]) -> type:
    """Create a simple struct-like class with named fields."""

    def __init__(self: Any, **kwargs: Any) -> None:
        for field in fields:
            if field not in kwargs:
                raise TypeError(f"Missing required field: {field!r}")
            setattr(self, field, kwargs[field])

    def __repr__(self: Any) -> str:
        attrs = ", ".join(f"{f}={getattr(self, f)!r}" for f in fields)
        return f"{name}({attrs})"

    def __eq__(self: Any, other: Any) -> bool:
        if type(self) is not type(other):
            return NotImplemented
        return all(getattr(self, f) == getattr(other, f) for f in fields)

    return type(
        name,
        (),
        {
            "_fields": tuple(fields),
            "__init__": __init__,
            "__repr__": __repr__,
            "__eq__": __eq__,
        },
    )


# Create struct classes dynamically
Point = make_struct("Point", ["x", "y"])
Color = make_struct("Color", ["r", "g", "b"])

p = Point(x=3, y=4)
c = Color(r=255, g=128, b=0)

print(f"{p!r}")
print(f"{c!r}")
print(f"Point._fields = {Point._fields}")
print(f"Point(x=3, y=4) == Point(x=3, y=4): {p == Point(x=3, y=4)}")

# Missing field raises TypeError
try:
    Point(x=1)
except TypeError as e:
    print(f"\nTypeError: {e}")

## `exec()` and `eval()`: Dynamic Code Execution

`eval()` evaluates a single expression and returns its value. `exec()` executes
arbitrary statements. Both are powerful but dangerous -- never use them with
untrusted input.

**When to use them**: Code generation in frameworks (like `@dataclass` does internally),
plugin systems with trusted code, and REPL-like tools.

**When NOT to use them**: Anything involving user-supplied strings (use `ast.literal_eval()`
for safe evaluation of literals).

In [ ]:
import ast
from typing import Any

# eval() evaluates an expression and returns the result
result = eval("2 ** 10 + len('hello')")
print(f"eval('2 ** 10 + len(\'hello\')') = {result}")

# eval() with a controlled namespace
safe_globals: dict[str, Any] = {"__builtins__": {}}  # Strip all builtins
safe_locals: dict[str, Any] = {"x": 10, "y": 20}
result = eval("x + y", safe_globals, safe_locals)
print(f"eval('x + y', restricted) = {result}")

# exec() executes statements (no return value)
namespace: dict[str, Any] = {}
exec(
    """
def greet(name: str) -> str:
    return f"Hello, {name}!"

result = greet("World")
""",
    namespace,
)
print(f"\nexec created function: {namespace['greet']}")
print(f"exec result: {namespace['result']!r}")

# SAFE alternative for parsing literals: ast.literal_eval
# Only allows: strings, bytes, numbers, tuples, lists, dicts, sets, bools, None
safe_data = ast.literal_eval("{'name': 'Alice', 'scores': [95, 87, 92]}")
print(f"\nast.literal_eval: {safe_data}")

# ast.literal_eval rejects anything dangerous
try:
    ast.literal_eval("__import__('os').system('echo hacked')")
except (ValueError, SyntaxError) as e:
    print(f"ast.literal_eval blocked: {type(e).__name__}: {e}")

## The `ast` Module: Parsing and Analyzing Python Source

The `ast` module parses Python source code into an Abstract Syntax Tree -- a tree of
node objects representing the structure of the code. This enables static analysis,
code transformation, and safe code inspection without executing anything.

In [ ]:
import ast

# Parse source code into an AST
source_code = """
def calculate(x: int, y: int) -> int:
    result = x * 2 + y
    return result

class Config:
    debug: bool = True
    max_retries: int = 3

    def validate(self) -> bool:
        return self.max_retries > 0
"""

tree = ast.parse(source_code)

# Walk the tree to find all function and class definitions
print("Top-level AST nodes:")
for node in ast.iter_child_nodes(tree):
    print(f"  {type(node).__name__}: ", end="")
    if isinstance(node, ast.FunctionDef):
        args = [arg.arg for arg in node.args.args]
        print(f"function '{node.name}' with args {args}")
    elif isinstance(node, ast.ClassDef):
        print(f"class '{node.name}'")
        for item in node.body:
            if isinstance(item, ast.FunctionDef):
                print(f"    method: '{item.name}'")
            elif isinstance(item, ast.AnnAssign) and isinstance(item.target, ast.Name):
                print(f"    field: '{item.target.id}'")

# ast.dump shows the full tree structure
simple_tree = ast.parse("x = 1 + 2")
print(f"\nAST dump of 'x = 1 + 2':")
print(ast.dump(simple_tree, indent=2))

In [ ]:
import ast


class FunctionAnalyzer(ast.NodeVisitor):
    """Walks an AST and collects information about function definitions."""

    def __init__(self) -> None:
        self.functions: list[dict[str, object]] = []

    def visit_FunctionDef(self, node: ast.FunctionDef) -> None:
        info: dict[str, object] = {
            "name": node.name,
            "args": [arg.arg for arg in node.args.args],
            "decorators": [
                ast.dump(d) for d in node.decorator_list
            ],
            "line": node.lineno,
            "has_return": any(
                isinstance(n, ast.Return) and n.value is not None
                for n in ast.walk(node)
            ),
            "num_statements": sum(
                1 for n in ast.walk(node)
                if isinstance(n, ast.stmt) and n is not node
            ),
        }
        self.functions.append(info)
        self.generic_visit(node)  # Continue visiting child nodes


# Analyze a code sample
code = """
def add(a, b):
    return a + b

def log(message):
    timestamp = get_time()
    formatted = f"[{timestamp}] {message}"
    print(formatted)

def noop():
    pass
"""

analyzer = FunctionAnalyzer()
analyzer.visit(ast.parse(code))

print(f"Found {len(analyzer.functions)} functions:\n")
for func in analyzer.functions:
    print(f"  {func['name']}({', '.join(func['args'])})")
    print(f"    Line: {func['line']}, Statements: {func['num_statements']}, Returns value: {func['has_return']}")

## Code Generation Patterns: Creating Functions at Runtime

Python frameworks like `dataclasses` and `attrs` use `exec()` internally to generate
methods with correct signatures. Here is how this pattern works and why it is
sometimes preferred over closures.

In [ ]:
from typing import Any


def make_init(fields: list[tuple[str, type, Any]]) -> Any:
    """Generate an __init__ method from a list of (name, type, default) triples.

    This is similar to what @dataclass does internally.
    """
    # Build the argument list
    args_parts: list[str] = ["self"]
    body_parts: list[str] = []
    local_ns: dict[str, Any] = {}

    for name, field_type, default in fields:
        if default is not ...:  # Ellipsis means no default
            default_name = f"_default_{name}"
            local_ns[default_name] = default
            args_parts.append(f"{name} = {default_name}")
        else:
            args_parts.append(name)

        body_parts.append(f"    self.{name} = {name}")

    args_str = ", ".join(args_parts)
    body_str = "\n".join(body_parts) if body_parts else "    pass"

    func_code = f"def __init__({args_str}):\n{body_str}"
    print(f"Generated code:\n{func_code}\n")

    exec(func_code, local_ns)
    return local_ns["__init__"]


# Generate an __init__ for a User class
init_method = make_init([
    ("name", str, ...),         # Required (no default)
    ("email", str, ...),        # Required
    ("active", bool, True),     # Optional with default
    ("role", str, "viewer"),    # Optional with default
])

# Attach the generated __init__ to a class
class User:
    __init__ = init_method

    def __repr__(self) -> str:
        return f"User(name={self.name!r}, email={self.email!r}, active={self.active}, role={self.role!r})"


u1 = User("Alice", "alice@example.com")
u2 = User("Bob", "bob@example.com", active=False, role="admin")

print(f"{u1!r}")
print(f"{u2!r}")

## Practical: ORM-Style Field Validator with Introspection

This practical example combines introspection, `__init_subclass__`, and dynamic
attribute access to build a mini ORM-style system. Fields are declared as class
annotations with `Field` descriptors, and the framework automatically generates
validation logic by inspecting the class.

In [ ]:
import inspect
from typing import Any, get_type_hints


class Field:
    """A descriptor that validates type and optional constraints."""

    def __init__(
        self,
        *,
        required: bool = True,
        default: Any = ...,
        min_value: float | None = None,
        max_value: float | None = None,
        min_length: int | None = None,
        max_length: int | None = None,
    ) -> None:
        self.required = required
        self.default = default
        self.min_value = min_value
        self.max_value = max_value
        self.min_length = min_length
        self.max_length = max_length
        # These are set by __set_name__
        self.name: str = ""
        self.storage_name: str = ""
        self.expected_type: type = object

    def __set_name__(self, owner: type, name: str) -> None:
        self.name = name
        self.storage_name = f"_field_{name}"

    def __get__(self, obj: Any, objtype: type | None = None) -> Any:
        if obj is None:
            return self
        return getattr(obj, self.storage_name, self.default)

    def __set__(self, obj: Any, value: Any) -> None:
        self.validate(value)
        setattr(obj, self.storage_name, value)

    def validate(self, value: Any) -> None:
        """Validate the value against all constraints."""
        if value is ... or value is None:
            if self.required:
                raise ValueError(f"Field {self.name!r} is required")
            return

        if self.expected_type is not object and not isinstance(value, self.expected_type):
            raise TypeError(
                f"Field {self.name!r}: expected {self.expected_type.__name__}, "
                f"got {type(value).__name__}"
            )

        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"Field {self.name!r}: {value} < min {self.min_value}")
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"Field {self.name!r}: {value} > max {self.max_value}")
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f"Field {self.name!r}: length {len(value)} < min {self.min_length}")
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(f"Field {self.name!r}: length {len(value)} > max {self.max_length}")


class ValidatedModel:
    """Base class that auto-configures Field descriptors using introspection."""

    def __init_subclass__(cls, **kwargs: Any) -> None:
        super().__init_subclass__(**kwargs)

        # Use get_type_hints to resolve annotations (including forward refs)
        hints = get_type_hints(cls)

        # Wire up each Field descriptor with its annotated type
        for attr_name, attr_value in cls.__dict__.items():
            if isinstance(attr_value, Field) and attr_name in hints:
                attr_value.expected_type = hints[attr_name]

    def __init__(self, **kwargs: Any) -> None:
        # Discover all Field descriptors in the class
        for attr_name in dir(type(self)):
            descriptor = getattr(type(self), attr_name, None)
            if isinstance(descriptor, Field):
                if attr_name in kwargs:
                    setattr(self, attr_name, kwargs[attr_name])
                elif descriptor.default is not ...:
                    setattr(self, attr_name, descriptor.default)
                elif descriptor.required:
                    raise ValueError(f"Missing required field: {attr_name!r}")

    def __repr__(self) -> str:
        fields = []
        for attr_name in dir(type(self)):
            descriptor = getattr(type(self), attr_name, None)
            if isinstance(descriptor, Field):
                value = getattr(self, attr_name)
                fields.append(f"{attr_name}={value!r}")
        return f"{type(self).__name__}({', '.join(fields)})"


print("ValidatedModel base class defined. Now define models...")

In [ ]:
# Define concrete models with validated fields

class Employee(ValidatedModel):
    name: str = Field(min_length=1, max_length=100)
    email: str = Field(min_length=5, max_length=255)
    age: int = Field(min_value=18, max_value=120)
    department: str = Field(required=False, default="unassigned")


class Product(ValidatedModel):
    sku: str = Field(min_length=3, max_length=20)
    price: float = Field(min_value=0.01)
    quantity: int = Field(min_value=0, default=0)


# Introspect the fields
print("Employee fields:")
for attr_name in sorted(dir(Employee)):
    descriptor = getattr(Employee, attr_name, None)
    if isinstance(descriptor, Field):
        constraints = []
        if descriptor.required:
            constraints.append("required")
        if descriptor.min_value is not None:
            constraints.append(f"min={descriptor.min_value}")
        if descriptor.max_value is not None:
            constraints.append(f"max={descriptor.max_value}")
        if descriptor.min_length is not None:
            constraints.append(f"minlen={descriptor.min_length}")
        if descriptor.max_length is not None:
            constraints.append(f"maxlen={descriptor.max_length}")
        print(f"  {attr_name}: {descriptor.expected_type.__name__} [{', '.join(constraints)}]")

# Create valid instances
emp = Employee(name="Alice Johnson", email="alice@example.com", age=30)
prod = Product(sku="WDG-001", price=29.99, quantity=100)

print(f"\n{emp!r}")
print(f"{prod!r}")

# Demonstrate validation errors
test_cases: list[tuple[str, dict[str, Any]]] = [
    ("Employee with empty name", {"name": "", "email": "a@b.com", "age": 25}),
    ("Employee too young", {"name": "Bob", "email": "b@c.com", "age": 15}),
    ("Product negative price", {"sku": "ABC", "price": -5.0}),
    ("Product wrong type for price", {"sku": "ABC", "price": "free"}),
]

print("\nValidation errors:")
for desc, kwargs in test_cases:
    try:
        if "sku" in kwargs:
            Product(**kwargs)
        else:
            Employee(**kwargs)
    except (TypeError, ValueError) as e:
        print(f"  {desc}: {type(e).__name__}: {e}")

## Summary

### Key Takeaways

| Tool | Purpose | Key Functions |
|------|---------|---------------|
| **`inspect`** | Examine live objects | `signature()`, `getsource()`, `getmembers()` |
| **`inspect` predicates** | Classify objects | `isclass()`, `isfunction()`, `ismethod()` |
| **`Signature`/`Parameter`** | Structured parameter info | `sig.parameters`, `param.kind`, `param.default` |
| **`type()` (3-arg)** | Create classes dynamically | `type(name, bases, namespace)` |
| **`exec()`/`eval()`** | Dynamic code execution | Use sparingly, never with untrusted input |
| **`ast`** | Parse and analyze source | `ast.parse()`, `ast.walk()`, `NodeVisitor` |
| **Code generation** | Generate methods at runtime | Used by `@dataclass`, `attrs`, ORMs |

### Best Practices
- Use `inspect.signature()` instead of manually parsing `__code__` attributes
- Prefer `ast.literal_eval()` over `eval()` for parsing data literals safely
- Use `type()` for simple dynamic class creation; use metaclasses for complex cases
- When using `exec()` for code generation, keep the generated code minimal and well-tested
- Combine `__init_subclass__` with `get_type_hints()` for annotation-driven frameworks
- The `ast.NodeVisitor` pattern is the idiomatic way to analyze Python source trees